# **PRÁCTICA FINAL - PREDICCIÓN DE DURACIÓN DE VIAJES Y DETECCIÓN DE EMERGENCIAS EN TWEETS**

**Asignatura:** Modelos no supervisados

**Fecha:** 09/06/2023

**Autores:** Mencía de Parias y Juan María Rotaeche

### **PARTE 0: Librerías**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import warnings
import torch

from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PowerTransformer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from transformers import AutoModel, AutoTokenizer

warnings.filterwarnings('ignore')
random_state = 100

### **PARTE 1: Predicción de duración de viajes**

##### **Ejercicio 1**

Realizar preprocesamiento de datos: imputar valores faltantes, transformar variables categóricas, estandarizar variables numéricas, etc. 
si lo consideras necesario para futuros modelos. Puede ser interesante intentar adaptar las variables que no siguan una distribución normal mediante técnicas de mapeado a gausianas como Power Transformers (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer). *(2puntos)*

En primer lugar leemos ambos DataFrames y con *head ()* nos hacemos una idea de la forma que tienen. Utilizamos también *shape* para ver la dimensionalidad de ambos.

In [2]:
uber_examples = pd.read_csv('uber_time_examples.csv')
uber_labels = pd.read_csv('uber_time_labels.csv')

In [3]:
print (uber_examples.shape, 
       uber_labels.shape)
print (uber_examples.head(), 
       uber_labels.head())

(400000, 12) (400000, 2)
   id       feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0  01-07 17:04:08          2       1.20        263        141  12.513054   
1   1  03-02 17:41:40          1       0.88        246         68   6.256527   
2   2  02-17 12:15:00          3       7.61         24         13  18.769581   
3   3  03-30 13:59:42          1       1.50        239        163   6.256527   
4   4  02-14 18:26:55          1       1.20        142        229   6.256527   

    feature_6  feature_7  feature_8  feature_9  feature_10  
0  297.430685  56.317405     405.20   0.408689  126.689773  
1  278.205127  27.160167     314.88  -0.256911  126.693467  
2   27.141964   5.192385      44.61  56.880789  126.615789  
3  270.288721  65.104518     403.50   1.218689  126.686311  
4  160.589952  91.465857     372.20   0.408689  126.689773      id  duration
0   0     455.0
1   1     413.0
2   2    1501.0
3   3     514.0
4   4     605.0


Ambas tablas tienen el mismo número de filas y la columna *id* para identificar los registros, por lo que el primer paso es unir los datos en base a esta columna.

In [4]:
uber = pd.merge (uber_labels, uber_examples, on = 'id')
uber.head()

,id,duration,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
0,0,455.0,01-07 17:04:08,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773
1,1,413.0,03-02 17:41:40,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467
2,2,1501.0,02-17 12:15:00,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789
3,3,514.0,03-30 13:59:42,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311
4,4,605.0,02-14 18:26:55,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773


Una vez tenemos los datos listos para empezar a trabajar, primero vamos a realizar un preprocesamiento de datos. Para ello, primero trabajaremos con los valores nulos si los hubiera; después analizaremos el tipo de variables que tenemos y si es necesario transformar alguna; y si realizamos una transformación a las variables que no siguen una distribución normal.

In [5]:
uber.isnull().sum()

id            0
duration      0
feature_0     0
feature_1     0
feature_2     0
feature_3     0
feature_4     0
feature_5     0
feature_6     0
feature_7     0
feature_8     0
feature_9     0
feature_10    0
dtype: int64

En todo nuestro conjunto de datos no hay ningún valor faltante.

In [6]:
uber.dtypes

id              int64
duration      float64
feature_0      object
feature_1       int64
feature_2     float64
feature_3       int64
feature_4       int64
feature_5     float64
feature_6     float64
feature_7     float64
feature_8     float64
feature_9     float64
feature_10    float64
dtype: object

In [7]:
uber['feature_0'] = pd.to_datetime(uber['feature_0'], format='%m-%d %H:%M:%S')

In [8]:
transformador= PowerTransformer(method='yeo-johnson')
uber['duration'] = transformador.fit_transform(uber[['duration']])

La única columna a la que conviene cambiarle el tipo de datos es a *feature_0* ya que muestra una fecha pero está en formato object.

Además, con el método Yeo-Johnson de PowerTransformer hemos normlaizado la variable objetivo: *duration*

##### **Ejercicio 2**

Crear nuevas características (features) que puedan mejorar el poder predictivo del modelo. *(1 punto)*

Tenemos una columna en formato tiempo, y puede ser interesante crear tres diferentes en base a esta para tener el dia, mes y hora de cada reserva de hora. Así podremos ver la evolución a lo largo de los meses o si en algún dia/a alguna hora los trayectos son más largos.

In [9]:
uber['day'] = uber['feature_0'].dt.day
uber['month'] = uber['feature_0'].dt.month
uber['hour'] = uber['feature_0'].dt.hour


In [10]:
uber = uber.drop(['feature_0'], axis=1) #elimino la columna de las fechas xq creo que ya no aporta info
uber = uber.drop(['id'], axis = 1)

Una vez creadas estas tres nuevas columnas, la columna *feature_0* en formato fecha no nos aporta mucha más información. Además, tenemos una columna: *id* que tampoco nos aporta información. Para mejorar la capacidad predictora del modelo vamos a eliminar ambas.

In [11]:
uber.nunique() 

duration       5592
feature_1        10
feature_2      2709
feature_3       231
feature_4       255
feature_5        10
feature_6       231
feature_7       255
feature_8     40197
feature_9      2709
feature_10     2709
day              31
month             4
hour             24
dtype: int64

In [12]:
uber = pd.get_dummies(uber, columns = ['month'])

Con *nunique()* obtenemos el número de valores únicos que toma cada variable. En este caso, hemos hecho un *get_dummies* de month que solo toma 4 valores diferentes.

##### **Ejercicio 3**

Seleccionar las características más relevantes para predecir la duración del viaje. Utilizar técnicas de selección de características basadas en una sola variable o SelectFromModel. Evitar Recursive feature elimination debido a su alto coste computacional.
Ver : (https://scikit-learn.org/stable/modules/feature_selection.html) *(2puntos)*

Como técnica de selección de caracterísitcas más importantes en relación a la variable objetivo hemos utilizado la correlación. Además hemos comprobado después con *SelectFromModel* para ver si nos daban los mismos resultados ya que una variable puede estar muy correlacionada con la objetivo de forma negativa y no daría como resultado que están altamente relacionadas cuando si que lo están.

In [14]:
# COMO TECNICA DE SELECCION DE CARACTERÍSTICAS BASADA EN UNA VARIABLE USAMOS CORRELACIÓN, FALTA DEFINIR THRESHOLD EN LA ULTIMA LINEA DE CODIGO
correlation_matrix = uber.corr()
correlation_with_target = correlation_matrix['duration'].abs().sort_values(ascending=False)
correlation_with_target

duration      1.000000
feature_2     0.551888
feature_10    0.551888
feature_7     0.080211
feature_4     0.080211
feature_8     0.071753
feature_6     0.058250
feature_3     0.058250
hour          0.040001
feature_1     0.018312
feature_5     0.018312
feature_9     0.016372
day           0.014297
month_3       0.012911
month_1       0.011181
month_12      0.007903
month_2       0.002041
Name: duration, dtype: float64

In [15]:
# Separar las características y la variable objetivo
X = uber.drop(['duration'], axis=1)
y = uber['duration']

# Crear un modelo clasificador basado en árboles
modelo = RandomForestRegressor()
modelo.fit(X, y)

importancias = modelo.feature_importances_

# Crear un diccionario de características y sus importancias
diccionario_importancias = dict(zip(X.columns, importancias))

# Imprimir las importancias de las características en orden descendente
for caracteristica, importancia in sorted(diccionario_importancias.items(), key=lambda x: x[1], reverse=True):
    print(f"{caracteristica}: {importancia}")

feature_10: 0.3653198412808319
feature_2: 0.1679752337924987
feature_9: 0.16287291163222803
feature_8: 0.06408406411244687
hour: 0.05704190970428596
day: 0.05166053683278204
feature_4: 0.024535798216441668
feature_7: 0.0243346619432212
feature_6: 0.022176420273611844
feature_3: 0.022140207636935107
feature_1: 0.008739662181853514
feature_5: 0.008647341081112344
month_1: 0.007176558429115445
month_3: 0.006656301324834602
month_2: 0.006458697315644591
month_12: 0.00017985424215624595


El resultado que tenemos es ligeramente diferente, así que nos quedamos con el de *SelecFromModel* para concluir que las características más importantes en relación a la variable objetivo son: *feature_10*, *feature_2* y *feature_9*

##### **Ejercicio 4**

Entrenar un modelo sencillo como base y medir su MAPE (Mean Absolute Percentage Error) en el conjunto de test. Luego, elegir y entrenar dos modelos más avanzados (por ejemplo, ensambladores, máquinas de soporte vectorial, modelos bayesianos, redes neuronales) y comparar sus MAPEs. *(2puntos)*

Para este ejercicio trabajaremos con una muestra de 20,000 filas del conjunto original de datos.

In [13]:
uber_sample = uber.sample(n=20000, random_state = random_state)

In [14]:
X = uber_sample.drop(['duration'], axis=1)
y = uber_sample['duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

En primer lugar, dividimos nuestro conjunto de datos en dos dataframes, uno que contiene todas las variables excepto la objetivo y otro solo con la variable objetivo. Dividimos ambos en conjuntos de test y entrenamiento.

No normalizamos, hemos probado y da mejores resultados sin normalizar.

In [15]:
# Entrenar un modelo de regresión lineal
modelo_base = LinearRegression()
modelo_base.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones_base = modelo_base.predict(X_test)

y_pred_inv = transformador.inverse_transform(predicciones_base.reshape(-1, 1))
y_test_inv = transformador.inverse_transform(y_test.values.reshape(-1, 1))

# Calcular el error al cuadrado
squared_error = (y_pred_inv - y_test_inv) ** 2

resultados = pd.DataFrame({'Duración Real': y_test_inv.flatten(),
                            'Duración Predicha': y_pred_inv.flatten(),
                            'Squared Error': squared_error.flatten()})

print(resultados)

mse_lr = mean_squared_error(resultados['Duración Real'], resultados['Duración Predicha'], squared=False)

# Calcular el MAPE utilizando las predicciones inversas
mape_lr = mean_absolute_percentage_error(transformador.inverse_transform(y_test_inv), y_pred_inv)
print(f"MAPE del modelo base: {mape_lr}")

      Duración Real  Duración Predicha  Squared Error
0             261.0         375.718319   1.316029e+04
1            1675.0        3317.384853   2.697428e+06
2             263.0         408.196204   2.108194e+04
3             515.0         486.138736   8.329726e+02
4             630.0         426.524474   4.140229e+04
...             ...                ...            ...
5995          233.0         404.139586   2.928876e+04
5996          283.0         363.237076   6.437988e+03
5997         1002.0         692.664550   9.568842e+04
5998          558.0         458.887796   9.823229e+03
5999          678.0         428.482465   6.225900e+04

[6000 rows x 3 columns]
MAPE del modelo base: 0.9993614372456142


In [16]:
# Entrenar un modelo de Random Forest (ensambladores)
modelo_rf = RandomForestRegressor()
modelo_rf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones_rf = modelo_rf.predict(X_test)

y_pred_rf_inv = transformador.inverse_transform(predicciones_rf.reshape(-1, 1))
y_test_rf_inv = transformador.inverse_transform(y_test.values.reshape(-1, 1))

# Calcular el error al cuadrado
squared_error_rf = (y_pred_rf_inv - y_test_rf_inv) ** 2

resultados_rf = pd.DataFrame({'Duración Real': y_test_rf_inv.flatten(),
                            'Duración Predicha': y_pred_rf_inv.flatten(),
                            'Squared Error': squared_error_rf.flatten()})

print(resultados_rf)

mse_rf = mean_squared_error(resultados_rf['Duración Real'], resultados_rf['Duración Predicha'], squared=False)

# Calcular el MAPE en el conjunto de prueba para Random Forest
mape_rf = mean_absolute_percentage_error(transformador.inverse_transform(y_test_rf_inv), y_pred_rf_inv)
print(f"MAPE del modelo Random Forest: {mape_rf}")

      Duración Real  Duración Predicha  Squared Error
0             261.0         230.616687     923.145725
1            1675.0        1783.396555   11749.813096
2             263.0         221.324508    1736.846649
3             515.0         500.685772     204.897115
4             630.0         500.989185   16643.790458
...             ...                ...            ...
5995          233.0         275.887453    1839.333622
5996          283.0         225.979009    3251.393416
5997         1002.0         921.832267    6426.865421
5998          558.0         382.673984   30739.211972
5999          678.0         597.240184    6522.147873

[6000 rows x 3 columns]
MAPE del modelo Random Forest: 0.9997873962386802


In [17]:
from sklearn.svm import SVR

# Crear un modelo de Máquinas de Soporte Vectorial (SVM)
modelo_svm = SVR(kernel='rbf', C=100, gamma=0.2, epsilon=1.1, max_iter=50000)

# Entrenar el modelo con los datos de entrenamiento
modelo_svm.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones_svm = modelo_svm.predict(X_test)
y_pred_svm_inv = transformador.inverse_transform(predicciones_svm.reshape(-1, 1))
y_test_svm_inv = transformador.inverse_transform(y_test.values.reshape(-1, 1))

# Calcular el error al cuadrado
squared_error_svm = (y_pred_svm_inv - y_test_svm_inv) ** 2

resultados_svm = pd.DataFrame({'Duración Real': y_test_svm_inv.flatten(),
                            'Duración Predicha': y_pred_svm_inv.flatten(),
                            'Squared Error': squared_error_svm.flatten()})

print(resultados_svm)

mse_svm = mean_squared_error(resultados_svm['Duración Real'], resultados_svm['Duración Predicha'], squared=False)

# Calcular el MAPE en el conjunto de prueba 
mape_svm = mean_absolute_percentage_error(transformador.inverse_transform(y_test_svm_inv), y_pred_svm_inv)
print(f"MAPE del modelo Random Forest: {mape_svm}")

      Duración Real  Duración Predicha  Squared Error
0             261.0         620.718925   1.293977e+05
1            1675.0         627.489652   1.097278e+06
2             263.0         504.047493   5.810389e+04
3             515.0         577.540711   3.911341e+03
4             630.0         627.489651   6.301851e+00
...             ...                ...            ...
5995          233.0         280.449850   2.251488e+03
5996          283.0         627.384461   1.186007e+05
5997         1002.0         627.489652   1.402580e+05
5998          558.0         615.855838   3.347298e+03
5999          678.0         627.487695   2.551493e+03

[6000 rows x 3 columns]
MAPE del modelo Random Forest: 0.9988707254904025


In [18]:
# Entrenar un modelo MLP (red neuronal)
modelo_mlp = MLPRegressor()
modelo_mlp.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones_mlp = modelo_mlp.predict(X_test)
y_pred_mlp_inv = transformador.inverse_transform(predicciones_mlp.reshape(-1, 1))
y_test_mlp_inv = transformador.inverse_transform(y_test.values.reshape(-1, 1))


# Calcular el error al cuadrado
squared_error_mlp = (y_pred_mlp_inv - y_test_mlp_inv) ** 2

resultados_mlp = pd.DataFrame({'Duración Real': y_test_mlp_inv.flatten(),
                            'Duración Predicha': y_pred_mlp_inv.flatten(),
                            'Squared Error': squared_error_mlp.flatten()})

print(resultados_mlp)

mse_mlp = mean_squared_error(resultados_mlp['Duración Real'], resultados_mlp['Duración Predicha'], squared=False)

# Calcular el MAPE en el conjunto de prueba 
mape_mlp = mean_absolute_percentage_error(transformador.inverse_transform(y_test_mlp_inv), y_pred_mlp_inv)
print(f"MAPE del modelo mlp: {mape_mlp}")

      Duración Real  Duración Predicha  Squared Error
0             261.0         352.694903   8.407955e+03
1            1675.0        4726.247361   9.310110e+06
2             263.0         369.431223   1.132761e+04
3             515.0         515.777477   6.044710e-01
4             630.0         395.166887   5.514659e+04
...             ...                ...            ...
5995          233.0         347.262867   1.305600e+04
5996          283.0         335.038170   2.707971e+03
5997         1002.0         918.455123   6.979746e+03
5998          558.0         404.020523   2.370968e+04
5999          678.0         504.174380   3.021535e+04

[6000 rows x 3 columns]
MAPE del modelo mlp: 0.9995186742767636


In [19]:
models = ['Linear Regression', 'Random Forest', 'SVM', 'MLP']
mse = [mse_lr, mse_rf, mse_svm, mse_mlp]
mapes = [mape_lr.round(4), mape_rf.round(4), mape_svm.round(4), mape_mlp.round(4)]
summary_df = pd.DataFrame({'Model':models, 'MSE':mse, 'MAPE':mapes})
summary_df.head()

,Model,MSE,MAPE
0,Linear Regression,4084.617801,0.9994
1,Random Forest,4084.352208,0.9998
2,SVM,4131.251089,0.9989
3,MLP,4700.546231,0.9995


La columna *MSE* muestra el error cuadrático medio obtenido por cada modelo. El error cuadrático medio es una métrica comúnmente utilizada para evaluar la precisión de los modelos de regresión. Cuanto menor sea el valor del MSE, mejor será el ajuste del modelo a los datos, en este caso el modelo que da mejor resultados en base al MSE es Random Forest.

La columna *MAPE* muestra el error porcentual absoluto medio obtenido por cada modelo. El error porcentual absoluto medio es otra métrica de evaluación común para los modelos de regresión. Cuanto menor sea el valor del MAPE, mejor será el ajuste del modelo a los datos. Teniendo en cuenta esta métrica el modelo más acertado es SVM.

##### **Ejercicio 5**

Optimizar los hiperparámetros de los dos últimos modelos utilizando validación cruzada (cross-validation) y comparar sus MAPEs. Elegir el mejor modelo basándose en estos resultados. *(2puntos)*

In [21]:
# Crear los DataFrames X y y a partir del df 'uber_sample'
X = uber_sample[['duration']]
y = uber_sample['duration']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar los datos de las variables predictoras
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

# Random Forest
random_forest = RandomForestRegressor()
random_forest.fit(X_train_scaled, y_train)

# Soporte Vectorial (SVM)
svm_model = SVR()
svm_model.fit(X_train_scaled, y_train)

# MLP (red neuronal)
mlp_model = MLPRegressor()
mlp_model.fit(X_train_scaled, y_train)

# Definir los hiperparámetros a explorar para cada modelo
linear_params = {}
random_forest_params = {'n_estimators': [10, 50, 100], 'max_depth': [None, 5, 10]}
svm_params = {'C': [1, 10, 100], 'gamma': [0.1, 0.01, 0.001]}
mlp_params = {'hidden_layer_sizes': [(10,), (50,), (100,)], 'activation': ['relu', 'tanh'], 'alpha': [0.0001, 0.001, 0.01]}

# Realizar la búsqueda de hiperparámetros con validación cruzada (GridSearchCV)
linear_model_cv = GridSearchCV(linear_model, linear_params, cv=5)
linear_model_cv.fit(X_train_scaled, y_train)

random_forest_cv = GridSearchCV(random_forest, random_forest_params, cv=5)
random_forest_cv.fit(X_train_scaled, y_train)

svm_model_cv = GridSearchCV(svm_model, svm_params, cv=5)
svm_model_cv.fit(X_train_scaled, y_train)

mlp_model_cv = GridSearchCV(mlp_model, mlp_params, cv=5)
mlp_model_cv.fit(X_train_scaled, y_train)

# Obtener los mejores modelos con los hiperparámetros optimizados
best_linear_model = linear_model_cv.best_estimator_
best_random_forest = random_forest_cv.best_estimator_
best_svm_model = svm_model_cv.best_estimator_
best_mlp_model = mlp_model_cv.best_estimator_

# Realizar las predicciones con los modelos optimizados
linear_predictions = best_linear_model.predict(X_test_scaled)
random_forest_predictions = best_random_forest.predict(X_test_scaled)
svm_predictions = best_svm_model.predict(X_test_scaled)
mlp_predictions = best_mlp_model.predict(X_test_scaled)

# Calcular los MSE y MAPE para cada modelo
linear_mse = mean_squared_error(y_test, linear_predictions)
linear_mape = mean_absolute_percentage_error(y_test, linear_predictions)

random_forest_mse = mean_squared_error(y_test, random_forest_predictions)
random_forest_mape = mean_absolute_percentage_error(y_test, random_forest_predictions)

svm_mse = mean_squared_error(y_test, svm_predictions)
svm_mape = mean_absolute_percentage_error(y_test, svm_predictions)

mlp_mse = mean_squared_error(y_test, mlp_predictions)
mlp_mape = mean_absolute_percentage_error(y_test, mlp_predictions)

# Crear el DataFrame con los resultados
results = pd.DataFrame({
    'Modelo Predictor': ['Regresión Lineal', 'Random Forest', 'SVM', 'MLP'],
    'MSE': [linear_mse.round(4), random_forest_mse.round(4), svm_mse, mlp_mse.round(4)],
    'MAPE': [linear_mape.round(4), random_forest_mape.round(4), svm_mape, mlp_mape.round(4)]
})

# Mostrar el DataFrame con los resultados
print(results)

   Modelo Predictor       MSE      MAPE
0  Regresión Lineal  0.000000  0.000000
1     Random Forest  0.000000  0.000100
2               SVM  0.000114  0.040016
3               MLP  0.000100  0.009800


### **PARTE 2: Detección de emergencias en tweets**

##### **Ejercicio 1:**

Extraer los embeddings del texto de los tweets utilizando un modelo pre-entrenado de Huggingface. *(1.5 puntos)*

En primer lugar, leemos la tabla *twitter* y imputamos *desconocido* en todos los valores faltantes.

In [23]:
twitter = pd.read_csv('twitter_emergency.csv')
twitter.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [24]:
print (twitter.isnull().sum(),
twitter.shape)

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64 (7613, 5)


In [25]:
twitter = twitter.fillna('desconocido')

Utilizamos el modelo *roberta-base* de HuggingFace para obtener los embeddigns.

In [ ]:
# Cargar el modelo y el tokenizer pre-entrenados
modelo = AutoModel.from_pretrained('roberta-base')
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Obtener los textos de los tweets
tweets = twitter['text'].tolist()

# Tokenizar los textos de los tweets
tokens = tokenizer(tweets, padding=True, truncation=True, max_length=128, return_tensors='pt')

# Obtener los embeddings de los tweets
with torch.no_grad():
    outputs = modelo(**tokens)

embeddings = outputs.last_hidden_state

# Los embeddings ahora contienen los vectores de representación del texto de los tweets
print(embeddings.shape)

##### **Ejercicio 2:**

Crear y entrenar una pequeña red neuronal que utilice los embeddings, la palabra clave (keyword) y la ubicación (location) para predecir si un tweet está relacionado con una emergencia o no. Gestionar los valores faltantes y agrupar las variables categóricas de manera adecuada. No es necesario realizar una optimización de hiperparámetros exhaustiva, pero se pueden realizar ajustes si se desea. 

NOTA: Si no se ha podido calcular los embeddings del ejercicio anterior, usar los que aparecen guardados como numpy.array en el fichero. *(1.5 puntos)*

In [11]:
embeddings = pd.read_pickle(r"C:\Users\menci\OneDrive - Colegio Universitario de Estudios Financieros (CUNEF)\CUNEF\4. CUARTO\2CUATRIMESTRE (NEWCASTLE)\Modelos supervisados\Practica-final\Practica-final\tweet_embeddings.pkl")

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Codificar las variables categóricas
encoder = LabelEncoder()
twitter['keyword'] = encoder.fit_transform(twitter['keyword'])
twitter['location'] = encoder.fit_transform(twitter['location'])

# Preparar los datos de entrada y salida
X = np.concatenate([embeddings, twitter[['keyword', 'location']].values], axis=1)
y = twitter['target'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear una función para construir el modelo de red neuronal
def create_model(optimizer_name):
    model = Sequential()
    model.add(Input(shape=X_train_scaled.shape[1:]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=0.001)
    else:
        optimizer = 'rmsprop'
        
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Crear el modelo utilizando KerasClassifier para ser compatible con GridSearchCV
model = KerasClassifier(build_fn=create_model)

# Definir los hiperparámetros a ajustar
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
    'optimizer_name': ['adam', 'rmsprop']
}

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_result.best_params_

# Crear y entrenar el modelo con los mejores hiperparámetros
best_model = create_model(best_params['optimizer_name'])
best_model.fit(X_train_scaled, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'])

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
print("MSE del modelo: %.2f" % mse)

# Calcular el error absoluto medio (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("MAE del modelo: %.2f" % mae)


Epoch 1/10
222/222 [==============================] - 2s 2ms/step - loss: 0.4925 - accuracy: 0.7827
Epoch 2/10
222/222 [==============================] - 0s 2ms/step - loss: 0.3467 - accuracy: 0.8511
Epoch 3/10
222/222 [==============================] - 0s 2ms/step - loss: 0.2929 - accuracy: 0.8820
Epoch 4/10
222/222 [==============================] - 0s 2ms/step - loss: 0.2405 - accuracy: 0.9023
Epoch 5/10
222/222 [==============================] - 0s 2ms/step - loss: 0.1953 - accuracy: 0.9243
Epoch 6/10
222/222 [==============================] - 0s 2ms/step - loss: 0.1802 - accuracy: 0.9288
Epoch 7/10
222/222 [==============================] - 1s 2ms/step - loss: 0.1533 - accuracy: 0.9420
Epoch 8/10
222/222 [==============================] - 0s 2ms/step - loss: 0.1267 - accuracy: 0.9533
Epoch 9/10
222/222 [==============================] - 1s 2ms/step - loss: 0.1105 - accuracy: 0.9640
Epoch 10/10
112/112 [==============================] - 0s 2ms/step - loss: 0.7572 - accuracy: 0.7856

In [34]:
from tensorflow.keras.optimizers import Adam
# Agrupar variables categóricas
twitter_encoded = pd.get_dummies(twitter, columns=['keyword', 'location'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(embeddings, twitter_encoded['target'], test_size=0.3, random_state=random_state)

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear el modelo de red neuronal
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("MSE del modelo: %.2f" % mse)

# Calcular el error absoluto medio (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("MAE del modelo: %.2f" % mae)


Epoch 1/10
167/167 [==============================] - 2s 6ms/step - loss: 0.5376 - accuracy: 0.7784 - val_loss: 0.3930 - val_accuracy: 0.8345
Epoch 2/10
167/167 [==============================] - 1s 4ms/step - loss: 0.4180 - accuracy: 0.8259 - val_loss: 0.3882 - val_accuracy: 0.8371
Epoch 3/10
167/167 [==============================] - 1s 4ms/step - loss: 0.3658 - accuracy: 0.8441 - val_loss: 0.3924 - val_accuracy: 0.8398
Epoch 4/10
167/167 [==============================] - 1s 3ms/step - loss: 0.3358 - accuracy: 0.8602 - val_loss: 0.3978 - val_accuracy: 0.8415
Epoch 5/10
167/167 [==============================] - 1s 3ms/step - loss: 0.3204 - accuracy: 0.8660 - val_loss: 0.4052 - val_accuracy: 0.8411
Epoch 6/10
167/167 [==============================] - 1s 3ms/step - loss: 0.2979 - accuracy: 0.8775 - val_loss: 0.4071 - val_accuracy: 0.8376
Epoch 7/10
167/167 [==============================] - 0s 3ms/step - loss: 0.2749 - accuracy: 0.8889 - val_loss: 0.4232 - val_accuracy: 0.8354
Epoch 

Entre los dos modelos de red neuronal creados para predecir si un tweet está relacionado con una emergencia o no en base a *keyword* y a *location*, funciona mejor el segundo en el que no se realiza optimización de hiperparámetros.